<span style="color:red">Abgegeben von (Name, Vorname):</span> 
Goxhufi, Driton

Immer griffbereit:
- Website: https://www.nltk.org/
- Buch: https://www.nltk.org/book/
- Module: https://www.nltk.org/py-modindex.html
- Beispiele: http://www.nltk.org/howto/

# Sätze: Tokenisierung  

Wörter treten gewöhnlich nicht isoliert auf, sondern gruppieren sich zu Chunks, Phrasen (oder Konsitutenten) und Sätzen. 

Bisher sind uns Worte und diese Gruppen im Brown Corpus fein säuberlich getrennt und voranalysiert begegnet, z.B. die Worte und Sätze im Brown Corpus:   

In [1]:
from nltk.corpus import brown

brown.tagged_sents()

[[('The', 'AT'), ('Fulton', 'NP-TL'), ('County', 'NN-TL'), ('Grand', 'JJ-TL'), ('Jury', 'NN-TL'), ('said', 'VBD'), ('Friday', 'NR'), ('an', 'AT'), ('investigation', 'NN'), ('of', 'IN'), ("Atlanta's", 'NP$'), ('recent', 'JJ'), ('primary', 'NN'), ('election', 'NN'), ('produced', 'VBD'), ('``', '``'), ('no', 'AT'), ('evidence', 'NN'), ("''", "''"), ('that', 'CS'), ('any', 'DTI'), ('irregularities', 'NNS'), ('took', 'VBD'), ('place', 'NN'), ('.', '.')], [('The', 'AT'), ('jury', 'NN'), ('further', 'RBR'), ('said', 'VBD'), ('in', 'IN'), ('term-end', 'NN'), ('presentments', 'NNS'), ('that', 'CS'), ('the', 'AT'), ('City', 'NN-TL'), ('Executive', 'JJ-TL'), ('Committee', 'NN-TL'), (',', ','), ('which', 'WDT'), ('had', 'HVD'), ('over-all', 'JJ'), ('charge', 'NN'), ('of', 'IN'), ('the', 'AT'), ('election', 'NN'), (',', ','), ('``', '``'), ('deserves', 'VBZ'), ('the', 'AT'), ('praise', 'NN'), ('and', 'CC'), ('thanks', 'NNS'), ('of', 'IN'), ('the', 'AT'), ('City', 'NN-TL'), ('of', 'IN-TL'), ('Atlant

Nun haben wir nicht immer das Glück, dass die Daten genau in dem Format vorliegen, das wir für die weitere Verarbeitung benötigen. Stattdessen kann es sein, dass wir zunächst Strings vor uns haben, die wir erst **tokenisieren** müssen:

> On a \\$50,000 mortgage of 30 years at 8%, the monthly payment wouldn't be \\$366.88.

> John Fitzgerald Kennedy wurde am 29. Mai 1917 als zweitältester Sohn von Joseph P. Kennedy und Rose Fitzgerald Kennedy in Brookline, Massachusetts geboren. Er stammte aus einer bedeutenden Familie: Sein Großvater mütterlicherseits war der demokratische Politiker John F. Fitzgerald. Seine jüngeren Brüder Robert – der 1968 ebenfalls einem Attentat zum Opfer fiel – und Edward spielten beide in der amerikanischen Geschichte des 20. Jahrhunderts als Politiker eine wesentliche Rolle.

Zwei Arten der Tokenisierung sind hier für uns relevant: 

- **Wort-Tokenisierung:** Die Identifikation von Worten im laufenden Text -- dazu zählen auch Zahlen, Satzzeichen etc.
- **Satz-Tokenisierung:** Die Identifikation von Sätzen im laufenden Text

Daneben gibt es aber viele weitere Arten der Tokenisierung, z.B. nach Zeilen, Tabs, [Themen](https://www.nltk.org/api/nltk.tokenize.html#module-nltk.tokenize.texttiling) usw.

**Man beachte:** Obwohl die Tokenisierung gerne als trivialer Vorverarbeitungsschritt angesehen wird, können sich die üblichen Tokenisierer überraschend deutlich unterscheiden (siehe [Dridan & Oepen 2012](https://www.aclweb.org/anthology/P12-2074)). Außerdem ist die Tokenisierung bis zu einem bestimmten Grad sprachabhängig.  

## Wort-Tokenisierung

Ziel bei der Wort-Tokenisierung ist die Identifikation von Worten und Satzzeichen im laufenden Text. Für obiges Beispiel würden wir also gerne eine Liste dieser Elemente erhalten:

    ['On', 'a', '$', '50,000', 'mortgage', ...]

Als erstes kommt hier wahrscheinlich die Stringmethode `split()` in den Sinn, mit der sich ein String an den Leerzeichen auftrennen lässt: 

In [2]:
s = "On a $50,000 mortgage of 30 years at 8%, the monthly payment wouldn't be $366.88."
print(s.split())

['On', 'a', '$50,000', 'mortgage', 'of', '30', 'years', 'at', '8%,', 'the', 'monthly', 'payment', "wouldn't", 'be', '$366.88.']


Das ist schon mal eine ganz gute Annäherung. Satzzeichen und Symbole wie `$` und `%` werden aber nicht als eigenständige Elemente erkannt, weil hier keine Leerzeichen als Separatoren verwendet werden.

Statt die Separatoren anzugeben, könnte man auch mit `re.findall()` spezifizieren, was überhaupt ein "Wort" sein soll:

In [3]:
import re
s = "On a $50,000 mortgage of 30 years at 8%, the monthly payment wouldn't be $366.88."

print(re.findall("(\$|\%|[^\%\s]+)",s)) 

['On', 'a', '$', '50,000', 'mortgage', 'of', '30', 'years', 'at', '8', '%', ',', 'the', 'monthly', 'payment', "wouldn't", 'be', '$', '366.88.']


Damit kommt man in unserem Beispiel bei den Symbolen `$` und `%` weiter, aber die Satzzeichen und Apostrophe bereiten immer noch Probleme, denn `,`, `'` und `.` können auch Bestandteile von Token sein. 

Die Lösung besteht darin, zuerst die fehlenden Separatoren einzufügen und erst dann die `split()`-Methode anzuwenden. 


In [4]:
s = "On a $50,000 mortgage of 30 years at 8%, the monthly payment wouldn't be $366.88."

s = re.sub('(\S+), ', r'\1 , ' , s)        # comma before whitespace
s = re.sub('\.$', ' .' , s)                # end period
s = re.sub('(\%|\$)',r' \1 ',s)            # percentage and Dollar symbol 
s = re.sub('(n\'t)(\.|\,| )', r' \1\2',s)  # negation contraction

print(s.split())

['On', 'a', '$', '50,000', 'mortgage', 'of', '30', 'years', 'at', '8', '%', ',', 'the', 'monthly', 'payment', 'would', "n't", 'be', '$', '366.88', '.']


So funktioniert im Prinzip auch der NLTK-Tokenisierer, der mittels `word_tokenize(string, language='english')` aufgerufen werden kann, wobei natürlich ein paar Substitutionsregeln mehr zum Einsatz kommen. 

In [5]:
from nltk.tokenize import word_tokenize
s = "On a $50,000 mortgage of 30 years at 8%, the monthly payment wouldn't be $366.88."
print(word_tokenize(s))

['On', 'a', '$', '50,000', 'mortgage', 'of', '30', 'years', 'at', '8', '%', ',', 'the', 'monthly', 'payment', 'would', "n't", 'be', '$', '366.88', '.']


## Satz-Tokenisierung

Die Wort-Tokenisierung (und insbesondere `word_tokenize()`) setzt in der Regel voraus, dass die Eingabestrings aus einzelnen Sätzen bestehen. Das sieht man daran, dass Punkte immer (und meistens auch nur dann) als Satzzeichen tokenisiert werden, wenn sie am Ende des Strings stehen. 

In [6]:
s1 = "John Fitzgerald Kennedy wurde am 29. Mai 1917 als zweitältester Sohn von Joseph P. Kennedy und Rose Fitzgerald Kennedy in Brookline, Massachusetts geboren." 
s2 = "John Fitzgerald Kennedy wurde am 29. Mai 1917 als zweitältester Sohn von Joseph P."
print(word_tokenize(s1, language='german'))
print(word_tokenize(s2, language='german'))

['John', 'Fitzgerald', 'Kennedy', 'wurde', 'am', '29.', 'Mai', '1917', 'als', 'zweitältester', 'Sohn', 'von', 'Joseph', 'P.', 'Kennedy', 'und', 'Rose', 'Fitzgerald', 'Kennedy', 'in', 'Brookline', ',', 'Massachusetts', 'geboren', '.']
['John', 'Fitzgerald', 'Kennedy', 'wurde', 'am', '29.', 'Mai', '1917', 'als', 'zweitältester', 'Sohn', 'von', 'Joseph', 'P', '.']


`word_tokenize()` erkennt im obigen Beispiel also nicht, dass `P.` eine Abkürzung ist. Die Unterscheidung zwischen Punkten in Abkürzungen und Punkten als Satzzeichen ist aber essentiell für die [Erkennung von Satzgrenzen](https://en.wikipedia.org/wiki/Sentence_boundary_disambiguation), d.h. für die **Satz-Tokenisierung**. 

Da Abkürzungen a priori keine festgelegte Form haben, liefern reguläre Ausdrücken alleine nicht immer zufriedenstellenden Ergebnisse. Stattdessen verwendet man üblicherweise überwachte und unüberwachte Lernverfahren, die über statistische Verteilungen in rohen oder schon analysierten Textdaten ermitteln, was eine Abkürzung ist und was nicht. Im Prinzip geht es dabei um Inferenzen wie: Tritt `P` statistisch signifikant mit `.` auf? Wenn ja, dann behandle `P.` als Abkürzung.  Man nennt diesen Ansatz deshalb auch **kollokationsbasiert**. 

Damit können über 90% der Satzgrenzen richtig erkannt werden [(Kiss & Strunk 2006)](https://www.aclweb.org/anthology/J06-4003/), wobei die tatsächliche Performanz stark von der Sprache, dem Test- und Trainingsset abhängt. NLTK enthält daher ein Werkzeug, um aus rohen Textdaten einen Satz-Tokenisierer zu lernen: 

     my_sent_tokenizer = PunktSentenceTokenizer(trainingData)
     
Dies ist insbesondere nützlich, wenn bestimmte Sprachen oder Textgenres analysiert werden sollen, für die es noch keinen angepassten Satz-Tokenisierer gibt.

Der vorgefertigte Satz-Tokenisierer im NLTK wird mit `sent_tokenizer(string,language='english')` aufgerufen. Man beachte, dass die Spracheinstellung einen erheblichen Einfluss auf das Ergebnis haben kann:

In [7]:
from nltk.tokenize import sent_tokenize
s = "John Fitzgerald Kennedy wurde am 29. Mai 1917 als zweitältester Sohn von Joseph P. Kennedy und Rose Fitzgerald Kennedy in Brookline, Massachusetts geboren." 
print(sent_tokenize(s,language='english'))
print(sent_tokenize(s,language='german'))

['John Fitzgerald Kennedy wurde am 29.', 'Mai 1917 als zweitältester Sohn von Joseph P. Kennedy und Rose Fitzgerald Kennedy in Brookline, Massachusetts geboren.']
['John Fitzgerald Kennedy wurde am 29. Mai 1917 als zweitältester Sohn von Joseph P. Kennedy und Rose Fitzgerald Kennedy in Brookline, Massachusetts geboren.']


Wenn wir auf diese Weise das ganze Eingangsbeispiel nach Sätzen tokenisieren, erhalten wir folgendes Ergebnis:

In [8]:
s = "John Fitzgerald Kennedy wurde am 29. Mai 1917 als zweitältester Sohn von Joseph P. Kennedy und Rose Fitzgerald Kennedy in Brookline, Massachusetts geboren. Er stammte aus einer bedeutenden Familie: Sein Großvater mütterlicherseits war der demokratische Politiker John F. Fitzgerald. Seine jüngeren Brüder Robert – der 1968 ebenfalls einem Attentat zum Opfer fiel – und Edward spielten beide in der amerikanischen Geschichte des 20. Jahrhunderts als Politiker eine wesentliche Rolle." 
print(sent_tokenize(s,language='german'))

['John Fitzgerald Kennedy wurde am 29. Mai 1917 als zweitältester Sohn von Joseph P. Kennedy und Rose Fitzgerald Kennedy in Brookline, Massachusetts geboren.', 'Er stammte aus einer bedeutenden Familie: Sein Großvater mütterlicherseits war der demokratische Politiker John F. Fitzgerald.', 'Seine jüngeren Brüder Robert – der 1968 ebenfalls einem Attentat zum Opfer fiel – und Edward spielten beide in der amerikanischen Geschichte des 20. Jahrhunderts als Politiker eine wesentliche Rolle.']


Es fällt auf, dass die Erkennung von Satzgrenzen bei `:` offensichtlich nicht so gut funktioniert wie beim Punkt.

## Zurück zur Wort-Disambiguierung: Das Lesk-Verfahren

Mit Wort- und Satz-Tokenisierern können wir nun ein weiteres "wissensbasiertes" Wort-Disambiguierungsverfahren implementieren, das auf [Michael Lesk](https://en.wikipedia.org/wiki/Mike_Lesk) zurückgeht. 

Die Idee des sogenannten **Lesk-Verfahrens** ist recht simpel: Seien $\Sigma$ die Bedeutungen eines Wortes $w$ in einem Kontext $K$. Diejenige Bedeutung in $\Sigma$, dessen Beschreibung (Glosse, Beispielverwendung) die größte Überschneidung mit $K$ hat, wird als Bedeutung von $w$ in $K$ ausgegeben. Kurz gesagt: $L(w,K) = \arg\max_{\sigma_1,...,\sigma_n} \sigma_i \cap K$

**Ein Beispiel:** Das Wort *bank* hat im Englischen die Bedeutungen `bank.n.01` und `bank.n.02` mit den folgenden beiden Glossen:

In [9]:
from nltk.corpus import wordnet as wn
print('bank.n.01: ' + wn.synset('bank.n.01').definition())
print('bank.n.02: ' + wn.synset('bank.n.02').definition())

bank.n.01: sloping land (especially the slope beside a body of water)
bank.n.02: a financial institution that accepts deposits and channels the money into lending activities


Wenn $K = $*The water washed away the sandy bank of the river*, dann würde der Lesk-Algorithmus die Bedeutung `bank.n.01` ausgeben, da die Überschneidung zwischen $K$ und `wn.synset('bank.n.01').definition()` abzüglich Stopwörter größer ist und *water* enthält.   

Es gibt viele mögliche Varianten des Lesk-Algorithmus. Der Kontext $K$ kann hier z.B. bestehen aus: 
- den Wortformen der Worttoken in $K$
- den möglichen Bedeutungen der Worttoken in $K$

Die Bedeutungen in $\Sigma$ können z.B. bestehen aus:
- den Definitionen oder den Beispielen der Synsets, oder beidem
- den Definitionen und Beispielen der durch Hyponymie direkt verbundenen Synsets (Banerjee & Pedersen 2002)


### <span style="color:red">Aufgaben I</span>

Als nächstes wollen wir den Lesk-Algorithmus implementieren und für die Evaluation wieder die Senseval-Daten für *interest* verwenden. 

D.h. wir brauchen wieder das Dictionary `SV_SENSE_MAP`, um die Synset-Namen bei der Umwandlung der Senseval-Daten zu aktualisieren:

In [10]:
# Copied from https://stackoverflow.com/a/16391584/6452961

# A map of SENSEVAL senses to WordNet 3.0 senses.
# SENSEVAL-2 uses WordNet 1.7, which is no longer installable on most modern
# machines and is not the version that the NLTK comes with.
# As a consequence, we have to manually map the following
# senses to their equivalent(s).
SV_SENSE_MAP = {
    "HARD1": ["difficult.a.01"],    # not easy, requiring great physical or mental
    "HARD2": ["hard.a.02",          # dispassionate
              "difficult.a.01"],
    "HARD3": ["hard.a.03"],         # resisting weight or pressure
    "interest_1": ["interest.n.01"], # readiness to give attention
    "interest_2": ["interest.n.03"], # quality of causing attention to be given to
    "interest_3": ["pastime.n.01"],  # activity, etc. that one gives attention to
    "interest_4": ["sake.n.01"],     # advantage, advancement or favor
    "interest_5": ["interest.n.05"], # a share in a company or business
    "interest_6": ["interest.n.04"], # money paid for the use of money
    "cord": ["line.n.18"],          # something (as a cord or rope) that is long and thin and flexible
    "formation": ["line.n.01","line.n.03"], # a formation of people or things one beside another
    "text": ["line.n.05"],                 # text consisting of a row of words written across a page or computer screen
    "phone": ["telephone_line.n.02"],   # a telephone connection
    "product": ["line.n.22"],       # a particular kind of product or merchandise
    "division": ["line.n.29"],      # a conceptual separation or distinction
    "SERVE12": ["serve.v.02"],       # do duty or hold offices; serve in a specific function
    "SERVE10": ["serve.v.06"], # provide (usually but not necessarily food)
    "SERVE2": ["serve.v.01"],       # serve a purpose, role, or function
    "SERVE6": ["service.v.01"]      # be used by; as of a utility
}

In [11]:
from nltk.corpus import senseval
interestGoldData = [[SV_SENSE_MAP[inst.senses[0]][0],inst.position,inst.context] 
                        for inst in senseval.instances('interest.pos')]
interestTestData = [['', inst[1], inst[2]] for inst in interestGoldData]

print(interestGoldData[0])
print(interestTestData[0])

['interest.n.04', 18, [('yields', 'NNS'), ('on', 'IN'), ('money-market', 'JJ'), ('mutual', 'JJ'), ('funds', 'NNS'), ('continued', 'VBD'), ('to', 'TO'), ('slide', 'VB'), (',', ','), ('amid', 'IN'), ('signs', 'VBZ'), ('that', 'IN'), ('portfolio', 'NN'), ('managers', 'NNS'), ('expect', 'VBP'), ('further', 'JJ'), ('declines', 'NNS'), ('in', 'IN'), ('interest', 'NN'), ('rates', 'NNS'), ('.', '.')]]
['', 18, [('yields', 'NNS'), ('on', 'IN'), ('money-market', 'JJ'), ('mutual', 'JJ'), ('funds', 'NNS'), ('continued', 'VBD'), ('to', 'TO'), ('slide', 'VB'), (',', ','), ('amid', 'IN'), ('signs', 'VBZ'), ('that', 'IN'), ('portfolio', 'NN'), ('managers', 'NNS'), ('expect', 'VBP'), ('further', 'JJ'), ('declines', 'NNS'), ('in', 'IN'), ('interest', 'NN'), ('rates', 'NNS'), ('.', '.')]]


<span style="color:red">A1:</span> Vervollständigen Sie die Funktion `lesk_disambiguate_interest` und implementieren Sie dabei eine Variante des Lesk-Algorithmus! Führen Sie für die Synset-Glossen und Synset-Beispielen die folgenden **Vorverarbeitungsschritte** aus: 

1. Wort- und Satz-Tokenisierung
2. Entfernung von Stopwörtern (`from nltk.corpus import stopwords`), Satzzeichen etc.
3. Lemmatisierung der übrigen Wortformen (`from nltk.stem import WordNetLemmatizer`)

In [104]:
synset_count = [[outsyns],[position],[context] for syns in interestGoldData]

SyntaxError: invalid syntax (<ipython-input-104-e89dedcfee61>, line 1)

In [79]:
wn.synset(interestGoldData[8][0])

Synset('interest.n.04')

In [102]:
len(interestGoldData[0][0])

13

In [25]:
wn.synset(interestGoldData[0][0]).definition()
wn.synset(interestGoldData[0][0]).definition()
for i in range(len(interestGoldData[:][0])):
    
[[syns.name(),syns.definition()] for syns in wn.synset(interestGoldData)]

AttributeError: 'list' object has no attribute 'lower'

In [103]:
wn.synset(interestGoldData[0][0]).definition()

'a fixed charge for borrowing money; usually a percentage of the amount borrowed'

In [12]:
from nltk.tokenize import word_tokenize

def lesk_disambiguate_interest(inst) :
    outsyns = inst[0]
    position = inst[1]
    context = inst[2]   # [('yields', 'NNS'), ('on', 'IN'), ('money-market', 'JJ'), ...]
    word_tokenize(s1, language='english')
    # Lösung A1
 
    
    return outsyns   

In [ ]:
# Test für A1

sumTrueDisambiguations = 0
sumFalseDisambiguations = 0

for i in range(len(interestTestData)) :
    if lesk_disambiguate_interest(interestTestData[i]) == interestGoldData[i][0] :
        sumTrueDisambiguations += 1
    else :
        sumFalseDisambiguations += 1

accuracyDisambiguations = sumTrueDisambiguations /(sumTrueDisambiguations + sumFalseDisambiguations)

print("Accuracy: {}".format(accuracyDisambiguations))